In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn import *
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()


In [ ]:
#(whole_market_train, whole_news_train) = env.get_training_data()
(market_train, news_train) = env.get_training_data()

In [ ]:
#market_train=whole_market_train.head(50000)
#news_train=whole_news_train.head(100000)
type(market_train)

In [ ]:
news_train.shape

In [ ]:
import datetime

market_train = market_train.loc[market_train['time'].dt.date>=datetime.date(2009,1,1)]
news_train = news_train.loc[news_train['time'].dt.date>=datetime.date(2009,1,1)]


market_train['close_open_ratio'] = np.abs(market_train['close']/market_train['open'])
threshold = 0.5
print('In %i lines price increases by 50%% or more in a day' %(market_train['close_open_ratio']>=1.5).sum())
print('In %i lines price decreases by 50%% or more in a day' %(market_train['close_open_ratio']<=0.5).sum())


market_train = market_train.loc[market_train['close_open_ratio'] < 1.5]
market_train = market_train.loc[market_train['close_open_ratio'] > 0.5]
market_train = market_train.drop(columns=['close_open_ratio'])


column_market = ['returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
column_raw = ['returnsClosePrevRaw1', 'returnsOpenPrevRaw1','returnsClosePrevRaw10', 'returnsOpenPrevRaw10']
for i in range(len(column_raw)):
    market_train[column_market[i]] = market_train[column_market[i]].fillna(market_train[column_raw[i]])
    
    
print('Removing outliers ...')
column_return = column_market + column_raw + ['returnsOpenNextMktres10']
orig_len = market_train.shape[0]
for column in column_return:
    market_train = market_train.loc[market_train[column]>=-2]
    market_train = market_train.loc[market_train[column]<=2]
new_len = market_train.shape[0]
rmv_len = np.abs(orig_len-new_len)
print('There were %i lines removed' %rmv_len)



print('Removing strange data ...')
orig_len = market_train.shape[0]
market_train = market_train[~market_train['assetCode'].isin(['PGN.N','EBRYY.OB'])]
#market_train = market_train[~market_train['assetName'].isin(['Unknown'])]
new_len = market_train.shape[0]
rmv_len = np.abs(orig_len-new_len)
print('There were %i lines removed' %rmv_len)



# Function to remove outliers
def remove_outliers(data_frame, column_list, low=0.02, high=0.98):
    for column in column_list:
        this_column = data_frame[column]
        quant_df = this_column.quantile([low,high])
        low_limit = quant_df[low]
        high_limit = quant_df[high]
        data_frame[column] = data_frame[column].clip(lower=low_limit, upper=high_limit)
    return data_frame



# Remove outlier
columns_outlier = ['takeSequence', 'bodySize', 'sentenceCount', 'wordCount', 'sentimentWordCount', 'firstMentionSentence','noveltyCount12H',\
                  'noveltyCount24H', 'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H', 'volumeCounts24H',\
                  'volumeCounts3D','volumeCounts5D','volumeCounts7D']
print('Clipping news outliers ...')
news_train = remove_outliers(news_train, columns_outlier)

In [ ]:
def preprocess_news(news_train):
    drop_list = [
        'audiences', 'subjects', 'assetName',
        'headline'
    ]
    news_train.drop(drop_list, axis=1, inplace=True)
    
    # Factorize categorical columns
    for col in ['headlineTag', 'provider', 'sourceId']:
        news_train[col], uniques = pd.factorize(news_train[col])
        del uniques
    
    # Remove {} and '' from assetCodes column
    news_train['assetCodes'] = news_train['assetCodes'].apply(lambda x: x[1:-1].replace("'", ""))
    return news_train

news_train = preprocess_news(news_train)

news_train.head(4)


In [ ]:
# import gc
# def unstack_asset_codes(news_train):
#     codes = []
#     indexes = []
#     for i, values in news_train['assetCodes'].iteritems():
#         #values=values.strip('{').strip('}').strip("'").strip("'")
#         explode = values.split(", ")
#         codes.extend(explode)
#         repeat_index = [int(i)]*len(explode)
#         indexes.extend(repeat_index)
        
#     index_df = pd.DataFrame({'news_index': indexes, 'assetCode': codes})
#     del codes, indexes
#     gc.collect()
#     return index_df

# index_df = unstack_asset_codes(news_train)

# index_df.head(5)


In [ ]:
# def merge_news_on_index(news_train, index_df):
#     news_train['news_index'] = news_train.index.copy()

#     # Merge news on unstacked assets
#     news_unstack = index_df.merge(news_train, how='left', on='news_index')
#     news_unstack.drop(['news_index', 'assetCodes'], axis=1, inplace=True)
#     return news_unstack

# news_unstack = merge_news_on_index(news_train, index_df)
# del news_train, index_df
# gc.collect()
# news_unstack.shape

In [ ]:
# news_unstack.shape

In [ ]:
import gc

def group_news(news_frame):
    news_frame['date'] = news_frame.time.dt.date  # Add date column
    
    aggregations = ['mean']
    gp = news_frame.groupby(['assetCodes', 'date']).agg(aggregations)
    gp.columns = pd.Index(["{}_{}".format(e[0], e[1]) for e in gp.columns.tolist()])
    gp.reset_index(inplace=True)
    # Set datatype to float32
    float_cols = {c: 'float32' for c in gp.columns if c not in ['assetCodes', 'date']}
    return gp.astype(float_cols)

news_agg = group_news(news_train)
# del news_unstack; 
gc.collect()
news_agg.head(3)

In [ ]:
news_agg.shape

In [ ]:
market_train['date'] = market_train.time.dt.date
#news_unstack['date'] = news_unstack.time.dt.date
df = market_train.merge(news_agg, how='left', left_on=['date', 'assetCode'], 
                            right_on=['date', 'assetCodes'])
del market_train, news_agg
gc.collect()
df.head(3)
df.shape
df.isna().sum()

In [ ]:
df = df.dropna(axis=0)
df.shape

In [ ]:
df.head(3)

In [ ]:
# def data_prep(market_train,news_unstack):
#     market_train.time = market_train.time.dt.date
#     #news_unstack.time = news_unstack.time.dt.hour
#     news_unstack.sourceTimestamp= news_unstack.sourceTimestamp.dt.hour
#     news_unstack.firstCreated = news_unstack.firstCreated.dt.date
    
#     #news_train['assetCodesLen'] = news_train['assetCodes'].map(lambda x: len(eval(x)))
#     #news_unstack['assetCode'] = news_unstack['assetCode'].map(lambda x: eval(x))
#     kcol = ['firstCreated', 'assetCode']
#     news_unstack = news_unstack.groupby(kcol, as_index=False).mean()
#     print("The shape after groub by is ",news_unstack.shape)
#     print("The shape of market train before is ",market_train.shape)
#     market_train = pd.merge(market_train, news_unstack, how='left', left_on=['time', 'assetCode'], right_on=['firstCreated', 'assetCode'])
#     print("The shape of market train after is ",market_train.shape)
#     lbl = {k: v for v, k in enumerate(market_train['assetCode'].unique())}
#     market_train['assetCodeT'] = market_train['assetCode'].map(lbl)
    
    
#     market_train = market_train.dropna(axis=0)
    
#     return market_train

In [ ]:
# def data_prep(market_train,news_train):
#     market_train.time = market_train.time.dt.date
#     news_train.time = news_train.time.dt.hour
#     news_train.sourceTimestamp= news_train.sourceTimestamp.dt.hour
#     news_train.firstCreated = news_train.firstCreated.dt.date
#     news_train['assetCodesLen'] = news_train['assetCodes'].map(lambda x: len(eval(x)))
#     news_train['assetCodes'] = news_train['assetCodes'].map(lambda x: list(eval(x))[0])
#     kcol = ['firstCreated', 'assetCodes']
#     news_train = news_train.groupby(kcol, as_index=False).mean()
#     market_train = pd.merge(market_train, news_train, how='left', left_on=['time', 'assetCode'], 
#                             right_on=['firstCreated', 'assetCodes'])
#     lbl = {k: v for v, k in enumerate(market_train['assetCode'].unique())}
#     market_train['assetCodeT'] = market_train['assetCode'].map(lbl)
    
    
#     market_train = market_train.dropna(axis=0)
    
#     return market_train

In [ ]:
#market_train = data_prep(market_train,news_unstack)

#print(list(market_train))

In [ ]:
up = df.returnsOpenNextMktres10 >= 0

In [ ]:
import pandas as pd


In [ ]:
fcol = [c for c in df if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'time_x', 'universe','sourceTimestamp']]

In [ ]:
X = df[fcol].values
up = up.values
r = df.returnsOpenNextMktres10.values
#print(market_train[fcol].shape)

In [ ]:
import numpy as np
mins = np.min(X, axis=0)
maxs = np.max(X, axis=0)
rng = maxs - mins
X = 1 - ((maxs - X) / rng)

In [ ]:
assert X.shape[0] == up.shape[0] == r.shape[0] 

In [ ]:
X_train, X_test, up_train, up_test, r_train, r_test\
= model_selection.train_test_split(X, up, r, test_size=0.25, random_state=0)

In [ ]:
# from sklearn.decomposition import PCA
# print("shape before pca is ",X_train.shape)
# Data_pca= PCA(n_components=10).fit(X_train)
# X_train_pca= Data_pca.transform(X_train)
# print("shape after pca is ",X_train_pca.shape)

# X_test_pca= Data_pca.transform(X_test)

In [ ]:
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.metrics import accuracy_score
# BoostModel= AdaBoostClassifier(n_estimators=300)
# BoostModel.fit(X_train,up_train)
# print(BoostModel.score(X_test,up_test))

In [ ]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# LDAmodelFitted= LinearDiscriminantAnalysis().fit(X_train, up_train)
# print(LDAmodelFitted.score(X_test,up_test))

In [ ]:
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# QDAmodelFitted= QuadraticDiscriminantAnalysis().fit(X_train, up_train)
# print(QDAmodelFitted.score(X_test,up_test))


In [ ]:
# from sklearn.linear_model import LogisticRegression

# LogRegModel= LogisticRegression( )
# LogRegModel.fit(X_train,up_train)


In [ ]:
from sklearn.metrics import accuracy_score
#LogRegModel.score(X_test,up_test)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score
# knn=KNeighborsClassifier(n_neighbors=100) 
# knn.fit(X_train, up_train)
# knn.score(X_test,up_test)

In [ ]:
from xgboost import XGBClassifier
import time

In [ ]:
xgb_up = XGBClassifier(n_jobs=4,n_estimators=250,max_depth=8,eta=0.1)

In [ ]:
xgb_up.fit(X_train,up_train)

In [ ]:
from sklearn.metrics import accuracy_score
#accuracy_score(xgb_up.predict(X_test),up_test)
xgb_up.score(X_test,up_test)

In [ ]:
# t = time.time()
# print('Fitting Up')
# xgb_up.fit(X_train,up_train)
# print(f'Done, time = {time.time() - t}')

In [ ]:
# from sklearn.metrics import accuracy_score
# #accuracy_score(xgb_up.predict(X_test),up_test)
# xgb_up.score(X_test,up_test)

In [ ]:
days = env.get_prediction_days()

In [ ]:
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
for (market_obs_df, news_obs_df, predictions_template_df) in (days):
    n_days +=1
    print(n_days,end=' ')
    t = time.time()
    #market_obs_df = data_prep(market_obs_df, news_obs_df)
    news_obs_df = preprocess_news(news_obs_df)
#   index_df = unstack_asset_codes(news_obs_df)
#   news_unstack = merge_news_on_index(news_obs_df, index_df)
    news_agg = group_news(news_obs_df)
    
    market_obs_df['date'] = market_obs_df.time.dt.date
    df = market_obs_df.merge(news_agg, how='left', left_on=['date', 'assetCode'], 
                            right_on=['date', 'assetCodes'])
    del market_obs_df, news_agg
    gc.collect()

    
    df = df[df.assetCode.isin(predictions_template_df.assetCode)]
    X_live = df[fcol].values
    X_live = 1 - ((maxs - X_live) / rng)
    prep_time += time.time() - t
    
    t = time.time()
    lp = xgb_up.predict_proba(X_live)
    prediction_time += time.time() -t
    
    t = time.time()
    confidence = 2* lp[:,1] -1
    preds = pd.DataFrame({'assetCode':df['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t

In [ ]:
env.write_submission_file()